# Import Data & Packages

In [1]:
import pandas as pd
import numpy as np
import datetime
from ast import literal_eval
import konlpy
from collections import Counter
import time
import networkx as nx
import matplotlib.pyplot as plt
import re
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import copy

In [200]:
df_whole=pd.read_json('amore_data_detected_bert.json')
df_whole['review_split_string']=df_whole['review_split'].copy()
df_whole['review_split']=df_whole['review_split_string'].apply(literal_eval)
df_whole['name']=df_whole[['brand','big_category','prod_nm']].agg('#'.join, axis=1)
df_whole['name_sub']=df_whole[['brand','big_category']].agg('#'.join, axis=1)

df_whole['name']=df_whole['name'].str.replace('ml', '')
df_whole['name']=df_whole['name'].str.replace('g', '')
df_whole['name']=df_whole['name'].str.replace('\d+', 'kk').str.replace('\d+', 'kk')
df_whole['name']=df_whole['name'].str.replace('kk매', '')
df_whole['name']=df_whole['name'].str.replace('kk호', '')
df_whole['name']=df_whole['name'].str.replace('kk', '')
df_whole['name']=df_whole['name'].str.replace('%', '')
df_whole['name']=df_whole['name'].str.replace('*', '')
df_whole['name']=df_whole['name'].str.replace('.', '')
df_whole['name']=df_whole['name'].apply(lambda x: re.sub('\[ ?(.*?)]', '', x,  flags=re.DOTALL))
df_whole['name']=df_whole['name'].apply(lambda x: re.sub('\( ?(.*?)\)', '', x,  flags=re.DOTALL))
df_whole['name']=df_whole['name'].apply(lambda x: x.strip())

df_whole['name']=df_whole['name'].str.replace('ml', '')
df_whole['name']=df_whole['name'].str.replace('g', '')

# Functions

In [6]:
def S1_number_func(x):
    return(sum([len(x[i].split(' ')) for i in range(len(x))]))

def S1_duplicate_func(x):
    return(len(set(x))/len(x))
    
def NVA(x):    
    total_words=[]
    for i, document in enumerate(list(set(x))):
        okt = konlpy.tag.Okt()
        clean_words = []
        for word in okt.pos(document, stem=True): 
            if word[1] in ['Noun', 'Verb', 'Adjective']: 
                clean_words.append(word[0])
        total_words.append(clean_words)
    result = []
    for x in total_words:
        result.extend(x)
    return result


def S1_diversity_func(NVA):  
    return(len(set(NVA))/len(NVA))
    

def S1_recency_func(x,max_date,window):
    """
    max=50 ( 50,49,,...1 )
    ----------------------------------------
    50,49,48,47,46 .... (max-0)~(max-window+1)
    49,48,47,46,45 .... (max-1)~(max-window+0)
    48,47,46,45,44 .... (max-2)~(max-window-1)
    ...
    5,4,3,2,1 ...,,,,,,. (max-45)~(max-window-44)
    ----------------------------------------
    max-45가 window가 될 때 까지
    """
    lower = max_date
    upper = max_date-window+1
    lower_sign=(x<=lower)
    upper_sign=(x>=upper)
    sign=lower_sign*upper_sign
    return sign

def s1_recency_list_func(prod_nm):
    tmp=S1_recency[S1_recency['prod_nm']==prod_nm]
    MA_list=[]
    for i in date_range:
        window_count = len(tmp['s1_recency'][tmp['s1_recency'].apply((lambda x: S1_recency_func(x,max_date-i,recency_window)))>0])
        window_count_mean = window_count/recency_window
        MA_list.append(window_count_mean)
    return MA_list

In [250]:
# def get_current_Kdays(df_,current_Y,current_M,current_D,K=14):
#     def recency_func(x,date):
#         return((date-x.date()).days)
#     df = copy.deepcopy(df_)
#     eval_date = datetime.date(current_Y,current_M,current_D)
#     df['recency']=df['date'].apply((lambda x: recency_func(x,eval_date)))
#     return_df=pd.DataFrame(df[(df['recency']<K)&(df['recency']>0)])
#     return eval_date, return_df

def get_current_k (df, Y, M, D, K = 60 ):
    from datetime import datetime
    eval_date = datetime(Y,M,D)
    from_date = datetime(Y,M,D) - timedelta(days = K )
    df =  df[ df['date'] >= from_date]
    df = df[ df['date'] < eval_date]
    return eval_date, df

def softmax(x):
    return (torch.exp(x) / torch.exp(x).sum()).flatten()


def fill_mtl_output(model, df):
    temp_cols=["overall({})".format(i) for i in [0,0.5,1]]+[c + '(0)' for c in cols1]+[c + '(0.5)' for c in cols1]+[c + '(1)' for c in cols1]+['ATT_'+c for c in cols1]
    df_temp=pd.DataFrame(index=df.index,columns=temp_cols)
    for ind in tqdm(df.index):
        
        review = torch.FloatTensor( df['embed_sents'][ind])
        num_sents = torch.tensor( len(review) )
        small_task= cat_task[ df['category'][ind] ]
        detect = df['mask'][ind]
        deteted_aspect= df.loc[ind,cols2][ df.loc[ind,cols2] == 1 ].index.values.tolist()
        deteted_aspect_ind = np.array( small_cat_indices[ small_task - 1 ] )[detect].tolist()
        deteted_aspect_target = np.array(cols1)[deteted_aspect_ind].tolist()
        model.eval()
        model.to('cpu')
        with torch.no_grad():
            out = model(review.unsqueeze(0), num_sents.unsqueeze(0))
        tg, ts, a1, a2, a3, a4, a5, a_g = out


        att_a = [a1[:3], a1[3:6],a1[6:], a2, a3, a4[:3], a4[4:], a5[:3], a5[3:]]

        overall_pred = dict( zip( ['5', '4', '1~3'], list(softmax(tg[small_task-1] )[[2,1,0]].data.numpy().round(3)) ) )

        df_temp.loc[ind,['overall(1)','overall(0.5)','overall(0)']]=list(overall_pred.values())

        calibrated_imp = a_g[small_task-1].flatten()[[df['mask'][ind]]].numpy() / sum(a_g[small_task-1].flatten()[[df['mask'][ind]]].numpy() )
        aspect_imps = dict( zip( deteted_aspect_target, calibrated_imp) )

        for id_, (aspect_ind, aspect_name,aspect_imp) in enumerate(zip(deteted_aspect_ind, deteted_aspect_target,aspect_imps.values())):
        
            a_index = np.where( (np.array(small_cat_indices[small_task-1]) == aspect_ind))[0][0]
            aspect_pred = dict(zip(['1','0.5', '0'], list(softmax(ts[small_task-1][a_index] )[[2,1,0]].data.numpy().round(3)) ))
            df_temp.loc[ind,aspect_name + '(1)']=aspect_pred['1']
            df_temp.loc[ind,aspect_name + '(0.5)']=aspect_pred['0.5']
            df_temp.loc[ind,aspect_name + '(0)']=aspect_pred['0']
            df_temp.loc[ind,'ATT_'+aspect_name]=aspect_imp
            
    df_temp = pd.concat([df_temp, df[['name','review_split_string','date']]],axis=1)
    return df_temp


# CBS I&V 계산

### 계산준비

In [9]:
model = torch.load('checkpoint39.pt',map_location='cpu')

In [15]:
cat_task = {'cat1': 1, 'cat2': 2,'cat3': 3,'cat4': 4,'cat5': 5,'cat6': 6,'cat7': 7,'cat9': 8,'cat10': 9}
cols1 = ['용량','민감성','향기','커버력','지속력','피부톤',
         '보습감','향','사용감','발림성','세정력', # 10
         '촉촉함','유분기','발색감','제형']#,'보습력']
cols2= df_whole.columns[25:48]
small_cat_indices = [[1, 10, 11],
                     [1, 6, 7],
                     [1, 8, 9],

                     [3, 4, 5],

                     [4, 5, 11, 12, 13, 14],

                     [4, 8, 13],
                     [4, 8, 13],

                     [2, 1, 0],
                     [2, 1, 0]]
cat_dict = dict(zip(['catA','catB','catC','catD','catE'], [3,3,6,3,3]))

In [ ]:
df_filled = fill_mtl_output(model, df_whole)

### 날짜

* 윈도우 : 60일
* 계산하는 날짜 : 6/1~7/31

In [294]:
def get_CBS(df_filled, start_date, end_date, window):
    def daterange(start_date, end_date):
        for n in range(int((end_date - start_date).days)):
            yield start_date + timedelta(n)
            
    lst = []
    for single_date in tqdm(daterange(start_date, end_date)):
        current_=single_date.strftime("%Y, %m, %d")
        current_=current_.split(',')

        date, df = get_current_k(df_filled, int(current_[0]),int(current_[1]),int(current_[2]),K=window)

        # (score1) Impression
        S1=df[['overall(0)','overall(0.5)','overall(1)','name']]
        S1['score']=(S1['overall(0.5)']*0.5 + S1['overall(1)']).astype('float')
        score1 = S1.groupby('name').mean()['score']

        # (score2) Variety
        df['detect_counts']=(df[df.columns[df.columns.str.contains('ATT_')]].isna()==False).sum(axis=1)
        df.fillna(0, inplace=True)
        emotion_mat = pd.concat([0.5*df[df.columns[(df.columns.str.contains('(0.5)'))&(~df.columns.str.contains('overall'))]], df[df.columns[(df.columns.str.contains('(1)'))&(~df.columns.str.contains('overall'))]]], axis=1)
        att_mat = pd.concat([df[df.columns[df.columns.str.contains('ATT_')]], df[df.columns[df.columns.str.contains('ATT_')]]], axis=1)
        df['variety_emotion']=np.multiply(np.array(emotion_mat), np.array(att_mat)).sum(axis=1)
        df['max_aspect']=df['name'].apply(lambda x: cat_dict[x.split('#')[1]])
        df['variety_calc']=df['detect_counts']*df['variety_emotion']/df['max_aspect']
        score2 = df.groupby('name').sum()['variety_calc']
        score2 = score2 / df.groupby('name').count().iloc[:,1]

        # (score3) Association


        # Final DataFrame
        df['brand'] = df['name'].apply(lambda x: x.split('#')[0])
        df['category'] = df['name'].apply(lambda x: x.split('#')[1])
        df['product'] = df['name'].apply(lambda x: x.split('#')[2])

        final=pd.DataFrame(df.groupby(['brand', 'category', 'product']).count().iloc[:,1]).reset_index()
        final.rename(columns={'overall(0.5)':'review_counts'}, inplace=True)
        final['date']=date
        final['score1']=score1.values
        final['score2']=score2.values
    #     final['score3']=score3.values

        lst.append(final)
        CBS = pd.concat([item for item in lst])
        
    return CBS

In [295]:
from datetime import date, timedelta

start_date = date(2021, 7, 1)
end_date = date(2021, 7, 10)

cbs = get_CBS(S23_org, start_date, end_date, 60)

2it [00:00, 14.25it/s]

(4945, 71)
(4977, 71)
(4960, 71)
(4933, 71)


6it [00:00, 15.24it/s]

(4918, 71)
(4918, 71)
(4899, 71)
(4863, 71)


9it [00:00, 15.56it/s]

(4838, 71)


In [296]:
cbs

,brand,category,product,review_counts,date,score1,score2
0,amore,catA,모이스춰 플럼핑 듀이 미스트,8,2021-07-01,0.535125,2.963116
1,amore,catA,더 에센셜 크림 플루이드,3,2021-07-01,0.583167,3.061066
2,amore,catA,라인에이징 코렉터,26,2021-07-01,0.556846,3.121141
3,amore,catA,모이스춰 플럼핑 넥타 크림,9,2021-07-01,0.558333,3.172879
4,amore,catA,모이스춰 플럼핑 듀이 미스트,19,2021-07-01,0.528711,3.205043
...,...,...,...,...,...,...,...
338,sulhwasoo,catC,에센셜 립세럼 스틱,4,2021-07-09,0.409000,1.365738
339,sulhwasoo,catE,동백윤모오일,2,2021-07-09,0.529500,2.768750
340,sulhwasoo,catE,바디로션 백은향,1,2021-07-09,0.523500,3.164900
341,sulhwasoo,catE,트래블키트 백은향,1,2021-07-09,0.562500,3.270000


In [291]:
cbs.groupby(['brand','category']).get_group(('amore','catA')).groupby('date').sum()

,review_counts,score1,score2
date,,,
2021-07-01,326,11.808573,69.174280
2021-07-02,326,11.806723,69.258934
2021-07-03,330,11.811404,69.216674
2021-07-04,328,11.807444,69.207605
2021-07-05,329,11.816733,69.681494
2021-07-06,331,11.821639,69.726581
2021-07-07,332,11.795578,69.754274
2021-07-08,326,11.786201,69.724435
2021-07-09,326,11.755814,69.816242


In [297]:
cbs.groupby(['brand','category']).get_group(('amore','catA')).groupby('date').sum()

,review_counts,score1,score2
date,,,
2021-07-01,326,11.808573,69.174280
2021-07-02,329,11.811253,69.191990
2021-07-03,331,11.809536,69.235915
2021-07-04,330,11.811404,69.216674
2021-07-05,332,11.817916,69.480010
2021-07-06,333,11.820439,69.721894
2021-07-07,336,11.817189,69.685858
2021-07-08,332,11.795578,69.754274
2021-07-09,331,11.764147,69.808179
